# Plant Disease recognition - Pattern Recognition Project

**Team Details:**
Hruday Chowdary Gurijala - S20210020278


Jayaraj Chippada - S20210020263


Sridhar Chunduri - S20210020266

#Motivation

As farmers and agriculture field are the important part of our life , farmers are the root level building blocks in the economy of any country . They work really heard for a whole season to grow a specific crop for survival of his family

Sometimes these crops on which he dedicated his whole 3-6 months to nurture these crops got disease as result of which they can't sell their crops on the price he was expecting

And He thinks if he knew these if he knew the plant disease before hand , he can use spefic pesticides and fertilizers to get over these disease

What if we can use deep learning techniques to help famers to know about specific disease , so that they can be ready before harvestifying their crops

# Dataset
We have 38 classes of plant disease images which contains 70295 images in training set and 17572 in valid set

Each class contains average of 1700-1800 number of images to work upon

Each image is of size= (256,256,3)

**!!! Using colab by Google is recommended or else install the libraries**

**needed using the code cell below. !!! RUN EACH CODE CELL ONE AFTER THE OTHER.**

In [ ]:
!pip install -r libraries.txt
# libraries.txt contains all libraries used and it can be found in the zip file.

**Upload the kaggle.json file in the zip file below**

In [ ]:
!pip install -q kaggle

from google.colab import files
files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

!unzip new-plant-diseases-dataset.zip

# Feature Extraction using Histograms

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import matplotlib.pyplot as plt
import os
from skimage.feature import graycomatrix
from skimage import io, color, feature
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pywt

In [2]:
def calculate_color_histogram(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert BGR to RGB (OpenCV uses BGR by default)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Calculate histograms for each color channel
    hist_red = cv2.calcHist([image_rgb], [0], None, [256], [0, 256])
    hist_green = cv2.calcHist([image_rgb], [1], None, [256], [0, 256])
    hist_blue = cv2.calcHist([image_rgb], [2], None, [256], [0, 256])

    return hist_red.flatten(), hist_green.flatten(), hist_blue.flatten()

In [3]:
def extract_features(image_path):
    # # Calculate color histogram
    red_hist, green_hist, blue_hist = calculate_color_histogram(image_path)
    all_features = np.concatenate((red_hist, green_hist, blue_hist))

    return all_features

In [ ]:
# Load your dataset and extract features
data_folder = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
BATCH_SIZE = 32
classes = os.listdir(data_folder)
X, y = [], []

In [ ]:
# Inside the loop where you read images
for class_name in classes:
    class_path = os.path.join(data_folder, class_name)
    images_in_class = os.listdir(class_path)

    for i in range(0, len(images_in_class), BATCH_SIZE):
        batch_images = images_in_class[i:i + BATCH_SIZE]
        batch_features = []

        for image_name in batch_images:
            image_path = os.path.join(class_path, image_name)

            # Check if the image can be read
            image = cv2.imread(image_path)
            if image is None or image.size == 0:
              print(f"Error: Unable to read or empty image at {image_path}")
              continue  # Skip to the next iteration

            features = extract_features(image_path)

            print(features)
            print(class_name)

            if features is not None:
                batch_features.append(features)

        if batch_features:
            # Concatenate features for the batch
            batch_features = np.vstack(batch_features)

            # Append features and labels to X and y
            X.extend(batch_features)
            y.extend([class_name] * len(batch_features))

In [ ]:

# Convert lists to numpy arrays
X_train = np.array(X)
y = np.array(y)

# Encode class labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y)

In [ ]:
# Load your dataset and extract features
data_folder_valid = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
BATCH_SIZE = 32
valid_classes = os.listdir(data_folder)
x_valid, y_valid = [], []

In [ ]:
for class_name in valid_classes:
    class_path = os.path.join(data_folder_valid, class_name)
    images_in_class = os.listdir(class_path)

    for i in range(0, len(images_in_class), BATCH_SIZE):
        batch_images = images_in_class[i:i + BATCH_SIZE]
        batch_features = []

        for image_name in batch_images:
            image_path = os.path.join(class_path, image_name)

            # Check if the image can be read
            image = cv2.imread(image_path)
            if image is None or image.size == 0:
              print(f"Error: Unable to read or empty image at {image_path}")
              continue  # Skip to the next iteration

            features = extract_features(image_path)

            print(features)
            print(class_name)

            if features is not None:
                batch_features.append(features)

        if batch_features:
            # Concatenate features for the batch
            batch_features = np.vstack(batch_features)

            # Append features and labels to X and y
            x_valid.extend(batch_features)
            y_valid.extend([class_name] * len(batch_features))


In [ ]:
# Convert lists to numpy arrays
X_test = np.array(x_valid)
y_valid = np.array(y_valid)

# Encode class labels
label_encoder = LabelEncoder()
y_test = label_encoder.fit_transform(y_valid)

**KNN Classifier:**

In [ ]:
# Create a KNN classifier with k=3
knn_classifier = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# Train the classifier
knn_classifier.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = knn_classifier.predict(X_test)
print(y_pred)

In [ ]:
# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix# Calculate confusion matrix
import seaborn as sns
cm = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(cm)

In [ ]:
# Plot the confusion matrix as a heatmap
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:

# Calculate correlation matrix
correlation_matrix = np.corrcoef(y_pred, y_test)
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", xticklabels=['y_probs', 'y_true'], yticklabels=['y_probs', 'y_true'])
plt.title("Correlation Matrix")
plt.show()

**Logistic Regression Model:**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [ ]:
# Create a logistic regression model for multi-class classification
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

In [ ]:
# Train the model on the training data
model.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_pred = model.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report_result)

In [ ]:
# Plot the confusion matrix as a heatmap
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:

# Calculate correlation matrix
correlation_matrix = np.corrcoef(y_pred, y_test)
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", xticklabels=['y_probs', 'y_true'], yticklabels=['y_probs', 'y_true'])
plt.title("Correlation Matrix")
plt.show()

**Random Forest Classifier:**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Create a Random Forest classifier for multiclass classification
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
# Train the model on the training data
model.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_pred = model.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report_result)

In [ ]:
# Plot the confusion matrix as a heatmap
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:

# Calculate correlation matrix
correlation_matrix = np.corrcoef(y_pred, y_test)
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", xticklabels=['y_probs', 'y_true'], yticklabels=['y_probs', 'y_true'])
plt.title("Correlation Matrix")
plt.show()

ANN:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [ ]:

# Encode class labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert class labels to one-hot encoded format
y_train_one_hot = to_categorical(y_train_encoded)
y_test_one_hot = to_categorical(y_test_encoded)

In [ ]:
# Build the ANN model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [ ]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train_encoded, epochs=50, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model on the validation data
eval_result = model.evaluate(X_test, y_test_encoded)

In [ ]:
## Display evaluation results
print(f'Loss: {eval_result[0]}, Accuracy: {eval_result[1]}')

In [ ]:
# Extract predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)


In [ ]:
# Convert one-hot encoded true labels to class labels
y_true = np.argmax(y_test_encoded)

# Print the evaluation results
print("\nEvaluation Results:")
print(f"Loss: {eval_result[0]}")
print(f"Accuracy: {eval_result[1]}")


In [ ]:
# Plot the confusion matrix as a heatmap
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:

# Calculate correlation matrix
correlation_matrix = np.corrcoef(y_pred, y_test)
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", xticklabels=['y_probs', 'y_true'], yticklabels=['y_probs', 'y_true'])
plt.title("Correlation Matrix")
plt.show()

**Decision Tree Classsifier:**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Initialize the decision tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)


In [ ]:
# Train the decision tree classifier
dt_classifier.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = dt_classifier.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
# Calculate correlation matrix
correlation_matrix = np.corrcoef(y_pred, y_test)
class_report = classification_report(y_test, y_pred)

# Print the evaluation results
print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

In [ ]:
# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", xticklabels=['y_probs', 'y_true'], yticklabels=['y_probs', 'y_true'])
plt.title("Correlation Matrix")
plt.show()

**Gaussian Naive Bayes classifier:**

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Initialize the Gaussian Naive Bayes classifier
nb_classifier = GaussianNB()


In [ ]:
# Train the classifier
nb_classifier.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)


In [ ]:
# Print the evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

In [ ]:
# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Calculate correlation matrix
correlation_matrix = np.corrcoef(y_pred, y_test)
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", xticklabels=['y_probs', 'y_true'], yticklabels=['y_probs', 'y_true'])
plt.title("Correlation Matrix")
plt.show()

CNN:

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import tensorflow
import tensorflow as tf
# from tensorflow.keras.applications import ResNet50
from tensorflow import keras
#from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,AveragePooling2D,Dense,Flatten,ZeroPadding2D,BatchNormalization,Activation,Add,Input,Dropout,GlobalAveragePooling2D
from keras.initializers import glorot_uniform
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
data_dir = "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = data_dir + "/train"
valid_dir = data_dir + "/valid"
test_path  = os.path.join("/content/test", "test")

In [ ]:
diseases = os.listdir(train_dir)
nums = {}
for disease in diseases:
    nums[disease] = len(os.listdir(train_dir + '/' + disease))
img_per_class = pd.DataFrame(nums.values(), index=nums.keys(), columns=["no. of images"])
img_per_class

In [ ]:
train_datagen= ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1/255.0,
    fill_mode='nearest',
    validation_split=0.1)
val_datagen= ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True,
    rescale=1/255.0,
    validation_split=0.1)
train= train_datagen.flow_from_directory(train_dir,batch_size=128,target_size=(210,210),color_mode='rgb',class_mode='categorical',seed=42)
valid=val_datagen.flow_from_directory(valid_dir,batch_size=128,target_size=(210,210),color_mode='rgb',class_mode='categorical')

In [ ]:
classes=list(train.class_indices.keys())
plt.figure(figsize=(20,20))
for X_batch, y_batch in train:
    for i in range(0,20):
        plt.subplot(5,5,i+1)
        plt.imshow(X_batch[i])
        plt.title(classes[np.where(y_batch[i]==1)[0][0]])
    plt.show()
    break

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32,3,activation="relu",padding="same",input_shape=(210,210,3)))
model.add(BatchNormalization())
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Conv2D(64,3,activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D())
model.add(BatchNormalization())
model.add(keras.layers.Conv2D(128,3,activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D())
model.add(BatchNormalization())

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(256,activation="relu"))
model.add(keras.layers.Dense(38,activation="softmax"))

model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train,validation_data=valid,epochs = 6)

In [ ]:
acc =history.history['accuracy']
val_acc =history.history['val_accuracy']
epochs = range(len(acc))
fig = plt.figure(figsize=(10,6))
plt.plot(epochs,acc,c="red",label="Training")
plt.plot(epochs,val_acc,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

In [ ]:
print("Train Accuracy  : {:.2f} %".format(history.history['accuracy'][-1]*100))

Mounting google drive to save models using .joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import joblib
joblib.dump(model,'/content/drive/MyDrive/trained_models/model1.joblib')

**Pre-Trained Model-> Resnet50:**

In [ ]:
path='/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
plt.figure(figsize=(70,70))
count=0
plant_names=[]
total_images=0
for i in os.listdir(path):
  count+=1
  plant_names.append(i)
  plt.subplot(7,7,count)

  images_path=os.listdir(path+"/"+i)
  print("Number of images of "+i+":",len(images_path),"||",end=" ")
  total_images+=len(images_path)

  image_show=plt.imread(path+"/"+i+"/"+images_path[0])

  plt.imshow(image_show)
  plt.xlabel(i)

  plt.xticks([])
  plt.yticks([])


print("Total number of images we have",total_images)

In [ ]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,AveragePooling2D,Dense,Flatten,ZeroPadding2D,BatchNormalization,Activation,Add,Input,Dropout,GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.initializers import glorot_uniform
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
base_model_tf=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=38)

In [ ]:
#Model building
base_model_tf.trainable=False

pt=Input(shape=(224,224,3))
func=tensorflow.cast(pt,tensorflow.float32)
x=preprocess_input(func) #This function used to zero-center each color channel wrt Imagenet dataset
model_resnet=base_model_tf(x,training=False)
model_resnet=GlobalAveragePooling2D()(model_resnet)
model_resnet=Dense(128,activation='relu')(model_resnet)
model_resnet=Dense(64,activation='relu')(model_resnet)
model_resnet=Dense(38,activation='softmax')(model_resnet)


model_main=Model(inputs=pt,outputs=model_resnet)
model_main.summary()

In [ ]:
#Image augmentation
train_datagen= ImageDataGenerator(shear_range=0.2,zoom_range=0.2,horizontal_flip=False,vertical_flip=False,fill_mode='nearest',width_shift_range=0.2,height_shift_range=0.2)

val_datagen=ImageDataGenerator()

path_train='/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

path_valid='/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

train= train_datagen.flow_from_directory(directory=path_train,batch_size=32,target_size=(224,224),color_mode='rgb',class_mode='categorical',seed=42)

valid=val_datagen.flow_from_directory(directory=path_valid,batch_size=32,target_size=(224,224),color_mode='rgb',class_mode='categorical')

In [ ]:
#CallBacks
es=EarlyStopping(monitor='val_accuracy',verbose=1,patience=7,mode='auto')
mc=ModelCheckpoint(filepath='/content',monitor='val_accuracy',verbose=1,save_best_only=True)
lr=ReduceLROnPlateau(monitor='val_accuracy',verbose=1,patience=5,min_lr=0.001)

In [ ]:
model_main.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#Training
model_main.fit(train,validation_data=valid,epochs=30,steps_per_epoch=200,verbose=1,callbacks=[mc,es,lr])

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(model_main.history.history['accuracy'],color='b',label='Training accuracy')
plt.plot(model_main.history.history['val_accuracy'],color='r',label='Validation accsuracy')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.title("accuracy graph")

In [ ]:
import pickle

# Save model using pickle
with open('model_main.pkl', 'wb') as file:
    pickle.dump(model, file)
#saved model

# Feature Extraction using Color Moments


In [5]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import matplotlib.pyplot as plt
import os
from skimage.feature import graycomatrix
from skimage import io, color, feature
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pywt
from scipy.stats import skew, kurtosis

In [6]:
def calculate_color_moments(image_path):
    # Read the image
    image = cv2.imread(image_path)
    # Convert BGR to RGB (OpenCV uses BGR by default)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Calculate color moments for each channel
    mean_values = np.mean(image_rgb, axis=(0, 1))
    variances = np.var(image_rgb, axis=(0, 1))
    skewness = skew(image_rgb.reshape(-1, 3), axis=0)
    kurt = kurtosis(image_rgb.reshape(-1, 3), axis=0)
    return np.concatenate([mean_values, variances, skewness, kurt])

In [7]:
def extract_features(image_path):
    # # Color moments
    color_moments = calculate_color_moments(image_path)
    return color_moments

In [ ]:
# Load your dataset and extract features
data_folder = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
BATCH_SIZE = 32
classes = os.listdir(data_folder)
X, y = [], []

# Inside the loop where you read images
for class_name in classes:
    class_path = os.path.join(data_folder, class_name)
    images_in_class = os.listdir(class_path)

    for i in range(0, len(images_in_class), BATCH_SIZE):
        batch_images = images_in_class[i:i + BATCH_SIZE]
        batch_features = []

        for image_name in batch_images:
            image_path = os.path.join(class_path, image_name)

            # Check if the image can be read
            image = cv2.imread(image_path)
            if image is None or image.size == 0:
                print(f"Error: Unable to read or empty image at {image_path}")
                continue  # Skip to the next iteration

            features = extract_features(image_path)
            print(features)
            print(class_name)

            if features is not None:
                batch_features.append(features)

        if batch_features:
            # Concatenate features for the batch
            batch_features = np.vstack(batch_features)
            # Append features and labels to X and y
            X.extend(batch_features)
            y.extend([class_name] * len(batch_features))

In [ ]:
# Convert lists to numpy arrays
X_train = np.array(X)
y = np.array(y)
# Encode class labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y)
# Load your dataset and extract features
data_folder_valid = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
BATCH_SIZE = 32
valid_classes = os.listdir(data_folder)
x_valid, y_valid = [], []

In [ ]:
# Inside the loop where you read images
for class_name in classes:
    class_path = os.path.join(data_folder_valid, class_name)
    images_in_class = os.listdir(class_path)

    for i in range(0, len(images_in_class), BATCH_SIZE):
        batch_images = images_in_class[i:i + BATCH_SIZE]
        batch_features = []

        for image_name in batch_images:
            image_path = os.path.join(class_path, image_name)

            # Check if the image can be read
            image = cv2.imread(image_path)
            if image is None or image.size == 0:
                print(f"Error: Unable to read or empty image at {image_path}")
                continue  # Skip to the next iteration

            features = extract_features(image_path)
            print(features)
            print(class_name)

            if features is not None:
                batch_features.append(features)

        # This block should be outside the inner loop
        if batch_features:
            # Concatenate features for the batch
            batch_features = np.vstack(batch_features)
            # Append features and labels to X and y
            x_valid.extend(batch_features)
            y_valid.extend([class_name] * len(batch_features))


In [ ]:
# Convert lists to numpy arrays
X_test = np.array(x_valid)
y_valid = np.array(y_valid)
# Encode class labels
label_encoder = LabelEncoder()
y_test = label_encoder.fit_transform(y_valid)

**KNN:**

In [ ]:
# Create a KNN classifier with k=3
knn_classifier = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# Train the classifier
knn_classifier.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = knn_classifier.predict(X_test)
print(y_pred)

In [ ]:
# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix# Calculate confusion matrix
import seaborn as sns
cm = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(cm)

In [ ]:
# Plot the confusion matrix as a heatmap
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Calculate correlation matrix
correlation_matrix = np.corrcoef(y_pred, y_test)
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", xticklabels=['y_probs', 'y_true'], yticklabels=['y_probs', 'y_true'])
plt.title("Correlation Matrix")
plt.show()

**Logistic Regression Model:**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [ ]:
# Create a logistic regression model for multi-class classification
model_cmlr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

In [ ]:
# Train the model on the training data
model_cmlr.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_pred = model_cmlr.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report_result)

In [ ]:
# Plot the confusion matrix as a heatmap
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Calculate correlation matrix
correlation_matrix = np.corrcoef(y_pred, y_test)
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", xticklabels=['y_probs', 'y_true'], yticklabels=['y_probs', 'y_true'])
plt.title("Correlation Matrix")
plt.show()

**Random Forest Classifier:**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Create a Random Forest classifier for multiclass classification
model_cmrf = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
# Train the model on the training data
model_cmrf.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_pred = model_cmrf.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report_result)

In [ ]:
# Plot the confusion matrix as a heatmap
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Calculate correlation matrix
correlation_matrix = np.corrcoef(y_pred, y_test)
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", xticklabels=['y_probs', 'y_true'], yticklabels=['y_probs', 'y_true'])
plt.title("Correlation Matrix")
plt.show()

**ANN:**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [ ]:
# Encode class labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert class labels to one-hot encoded format
y_train_one_hot = to_categorical(y_train_encoded)
y_test_one_hot = to_categorical(y_test_encoded)

In [ ]:
# Build the ANN model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [ ]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train_encoded, epochs=90, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model on the validation data
eval_result = model.evaluate(X_test, y_test_encoded)

In [ ]:
## Display evaluation results
print(f'Loss: {eval_result[0]}, Accuracy: {eval_result[1]}')

In [ ]:
# Extract predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
# Convert one-hot encoded true labels to class labels
y_true = np.argmax(y_test_encoded)

# Print the evaluation results
print("\nEvaluation Results:")
print(f"Loss: {eval_result[0]}")
print(f"Accuracy: {eval_result[1]}")

In [ ]:
# Plot the confusion matrix as a heatmap
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Calculate correlation matrix
correlation_matrix = np.corrcoef(y_pred, y_test)
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", xticklabels=['y_probs', 'y_true'], yticklabels=['y_probs', 'y_true'])
plt.title("Correlation Matrix")
plt.show()

**Decision Tree Classifier:**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Initialize the decision tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

In [ ]:
# Train the decision tree classifier
dt_classifier.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = dt_classifier.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
# Calculate correlation matrix
correlation_matrix = np.corrcoef(y_pred, y_test)
class_report = classification_report(y_test, y_pred)

# Print the evaluation results
print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

In [ ]:
# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", xticklabels=['y_probs', 'y_true'], yticklabels=['y_probs', 'y_true'])
plt.title("Correlation Matrix")
plt.show()

**Gaussian Naive Bayes Classifier:**

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Initialize the Gaussian Naive Bayes classifier
nb_classifier = GaussianNB()

In [ ]:
# Train the classifier
nb_classifier.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [ ]:
# Print the evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

In [ ]:
# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Calculate correlation matrix
correlation_matrix = np.corrcoef(y_pred, y_test)
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", xticklabels=['y_probs', 'y_true'], yticklabels=['y_probs', 'y_true'])
plt.title("Correlation Matrix")
plt.show()

**resnet50(Pre-Trained Model):**

In [ ]:
path='/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
plt.figure(figsize=(70,70))
count=0
plant_names=[]
total_images=0
for i in os.listdir(path):
  count+=1
  plant_names.append(i)
  plt.subplot(7,7,count)

  images_path=os.listdir(path+"/"+i)
  print("Number of images of "+i+":",len(images_path),"||",end=" ")
  total_images+=len(images_path)

  image_show=plt.imread(path+"/"+i+"/"+images_path[0])

  plt.imshow(image_show)
  plt.xlabel(i)

  plt.xticks([])
  plt.yticks([])


print("Total number of images we have",total_images)

In [ ]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,AveragePooling2D,Dense,Flatten,ZeroPadding2D,BatchNormalization,Activation,Add,Input,Dropout,GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.initializers import glorot_uniform
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
base_model_tf=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=38)

In [ ]:
#Model building
base_model_tf.trainable=False

pt=Input(shape=(224,224,3))
func=tensorflow.cast(pt,tensorflow.float32)
x=preprocess_input(func) #This function used to zero-center each color channel wrt Imagenet dataset
model_resnet=base_model_tf(x,training=False)
model_resnet=GlobalAveragePooling2D()(model_resnet)
model_resnet=Dense(128,activation='relu')(model_resnet)
model_resnet=Dense(64,activation='relu')(model_resnet)
model_resnet=Dense(38,activation='softmax')(model_resnet)


model_main=Model(inputs=pt,outputs=model_resnet)
model_main.summary()

In [ ]:
#Image augmentation
train_datagen= ImageDataGenerator(shear_range=0.2,zoom_range=0.2,horizontal_flip=False,vertical_flip=False,fill_mode='nearest',width_shift_range=0.2,height_shift_range=0.2)

val_datagen=ImageDataGenerator()

path_train='/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

path_valid='/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

train= train_datagen.flow_from_directory(directory=path_train,batch_size=32,target_size=(224,224),color_mode='rgb',class_mode='categorical',seed=42)

valid=val_datagen.flow_from_directory(directory=path_valid,batch_size=32,target_size=(224,224),color_mode='rgb',class_mode='categorical')

In [ ]:
#CallBacks
es=EarlyStopping(monitor='val_accuracy',verbose=1,patience=7,mode='auto')
mc=ModelCheckpoint(filepath='/content',monitor='val_accuracy',verbose=1,save_best_only=True)
lr=ReduceLROnPlateau(monitor='val_accuracy',verbose=1,patience=5,min_lr=0.001)

In [ ]:
model_main.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#Training
model_main.fit(train,validation_data=valid,epochs=6,steps_per_epoch=200,verbose=1,callbacks=[mc,es,lr])

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(model_main.history.history['accuracy'],color='b',label='Training accuracy')
plt.plot(model_main.history.history['val_accuracy'],color='r',label='Validation accsuracy')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.title("accuracy graph")

**CNN:**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import tensorflow
import tensorflow as tf
# from tensorflow.keras.applications import ResNet50
from tensorflow import keras
#from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,AveragePooling2D,Dense,Flatten,ZeroPadding2D,BatchNormalization,Activation,Add,Input,Dropout,GlobalAveragePooling2D
from keras.initializers import glorot_uniform
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
data_dir = "/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = data_dir + "/train"
valid_dir = data_dir + "/valid"
test_path  = os.path.join("/content/test", "test")

In [ ]:
diseases = os.listdir(train_dir)
nums = {}
for disease in diseases:
    nums[disease] = len(os.listdir(train_dir + '/' + disease))
img_per_class = pd.DataFrame(nums.values(), index=nums.keys(), columns=["no. of images"])
img_per_class

In [ ]:
train_datagen= ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1/255.0,
    fill_mode='nearest',
    validation_split=0.1)
val_datagen= ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True,
    rescale=1/255.0,
    validation_split=0.1)
train= train_datagen.flow_from_directory(train_dir,batch_size=128,target_size=(210,210),color_mode='rgb',class_mode='categorical',seed=42)
valid=val_datagen.flow_from_directory(valid_dir,batch_size=128,target_size=(210,210),color_mode='rgb',class_mode='categorical')

In [ ]:
classes=list(train.class_indices.keys())
plt.figure(figsize=(20,20))
for X_batch, y_batch in train:
    for i in range(0,20):
        plt.subplot(5,5,i+1)
        plt.imshow(X_batch[i])
        plt.title(classes[np.where(y_batch[i]==1)[0][0]])
    plt.show()
    break

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32,3,activation="relu",padding="same",input_shape=(210,210,3)))
model.add(BatchNormalization())
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Conv2D(64,3,activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D())
model.add(BatchNormalization())
model.add(keras.layers.Conv2D(128,3,activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D())
model.add(BatchNormalization())

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(256,activation="relu"))
model.add(keras.layers.Dense(38,activation="softmax"))

model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train,validation_data=valid,epochs = 6)

In [ ]:
acc =history.history['accuracy']
val_acc =history.history['val_accuracy']
epochs = range(len(acc))
fig = plt.figure(figsize=(10,6))
plt.plot(epochs,acc,c="red",label="Training")
plt.plot(epochs,val_acc,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

In [ ]:
print("Train Accuracy  : {:.2f} %".format(history.history['accuracy'][-1]*100))

As In formed in the evaluation, the stats of the cnn model is compared to other models in the project report.The code for the cnn model can be found in zip file.